In [82]:
from keras.layers import LSTM, Dense, Input, Dropout
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import pandas as pd
from dataclasses import dataclass, asdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [83]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [84]:
@dataclass
class modelConfig:
    ticker: str = 'NVDA'
    time_step: int = 5
    epochs: int = 20
    optimizer: str = 'adam'
    batch_size: int = 15
    learning_rate: float = 0.05
    nn_activation: str = 'relu'
    nn_max_units: int = 100
    nn_layers: int = 2
    nn_return_sequences: bool = True
    loss: str = 'mean_squared_error'
    dropout: bool = True
    dropout_value: float = 0.2
    period: str = '3mo'

    def get_parameters(self) -> dict:
        return asdict(self)

In [85]:
def create_dataset(scaled_data) -> np:
    X, y = [], []
    for i in range(len(scaled_data) - 5 - 1):
        X.append(scaled_data[i:(i + 5), 0])
        y.append(scaled_data[i + 5, 0])
    return np.array(X), np.array(y)

In [86]:
def read_yfinance(ticker: str, period: str) -> pd.DataFrame:
    nvda: pd.DataFrame = yf.Ticker(ticker).history(period)
    return pd.DataFrame(nvda['Close'])

In [87]:
ticker = 'NVDA'
period = '3mo'

In [88]:
data : pd.DataFrame = read_yfinance(ticker, period)

## Not working

In [89]:
data

,Close
Date,
2024-08-21 00:00:00-04:00,128.489014
2024-08-22 00:00:00-04:00,123.729416
2024-08-23 00:00:00-04:00,129.358932
2024-08-26 00:00:00-04:00,126.449181
2024-08-27 00:00:00-04:00,128.289032
...,...
2024-11-14 00:00:00-05:00,146.759995
2024-11-15 00:00:00-05:00,141.979996
2024-11-18 00:00:00-05:00,140.149994


In [90]:
X, y = [], []

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [91]:
X

array([[[0.55728349],
        [0.45394606],
        [0.57617062],
        [0.5129959 ],
        [0.55294161]],

       [[0.45394606],
        [0.57617062],
        [0.5129959 ],
        [0.55294161],
        [0.49454292]],

       [[0.57617062],
        [0.5129959 ],
        [0.55294161],
        [0.49454292],
        [0.32043248]],

       [[0.5129959 ],
        [0.55294161],
        [0.49454292],
        [0.32043248],
        [0.35907556]],

       [[0.55294161],
        [0.49454292],
        [0.32043248],
        [0.35907556],
        [0.11223818]],

       [[0.49454292],
        [0.32043248],
        [0.35907556],
        [0.11223818],
        [0.0733781 ]],

       [[0.32043248],
        [0.35907556],
        [0.11223818],
        [0.0733781 ],
        [0.09508766]],

       [[0.35907556],
        [0.11223818],
        [0.0733781 ],
        [0.09508766],
        [0.        ]],

       [[0.11223818],
        [0.0733781 ],
        [0.09508766],
        [0.        ],
        [0.07902

In [ ]:
model = Sequential()
model.add(Input(shape=(X.shape[1], 1)))
model.add(LSTM(units=50, activation='relu', return_sequences=True))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=False)

In [94]:
config = modelConfig()

In [95]:
config.get_parameters

<bound method modelConfig.get_parameters of modelConfig(ticker='NVDA', time_step=5, epochs=20, optimizer='adam', batch_size=15, learning_rate=0.05, nn_activation='relu', nn_max_units=100, nn_layers=2, nn_return_sequences=True, loss='mean_squared_error', dropout=True, dropout_value=0.2, period='3mo')>

In [81]:
config.model_evaluated = 0.34009817242622375

In [101]:
model.fit(X_train, y_train, epochs=5, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.3966 - mae: 0.5632 - mse: 0.3966 - val_loss: 0.9293 - val_mae: 0.9618 - val_mse: 0.9293
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3739 - mae: 0.5541 - mse: 0.3739 - val_loss: 0.8517 - val_mae: 0.9207 - val_mse: 0.8517
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3253 - mae: 0.5005 - mse: 0.3253 - val_loss: 0.7821 - val_mae: 0.8819 - val_mse: 0.7821
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3231 - mae: 0.5074 - mse: 0.3231 - val_loss: 0.7155 - val_mae: 0.8430 - val_mse: 0.7155
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3250 - mae: 0.5125 - mse: 0.3250 - val_loss: 0.6481 - val_mae: 0.8014 - val_mse: 0.6481


In [103]:
results = model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.6481 - mae: 0.8014 - mse: 0.6481


In [104]:
results

[0.6481223702430725, 0.8014066815376282, 0.6481223106384277]

In [72]:
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
